In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [2]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()

x_train = x_train.reshape((x_train.shape[0], -1)).astype('float32') / 255.0
x_test  = x_test.reshape((x_test.shape[0], -1)).astype('float32') / 255.0